In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from micron2.clustering import Autoencoder, train_AE_simCLR
from micron2.data import stream_dataset

import tqdm.auto as tqdm

outdir = 'trained_simclr'

In [2]:
import h5py
f = h5py.File('/dev/shm/dataset.hdf5', 'r')
print(f['meta/channel_names'][:])
use_channels = [b.decode('UTF-8') for b in f['meta/channel_names'][:]]
# fn = lambda x,y: print(x)
print(f.keys())
print(f['cells'].keys())
f.close()

[b'DAPI' b'OX40L' b'CD45' b'CD20' b'CD134' b'CD68' b'CD31' b'CD103'
 b'HLA-DR' b'CXCR5' b'IgG' b'CD3e' b'Ki-67' b'LAG3' b'CXCL13' b'IgA'
 b'CD89' b'PNaD' b'PD-L1' b'PD-1' b'CD11c' b'CD80' b'CD69' b'TIM3'
 b'CD45RO' b'CD40LG' b'FOXP3' b'CD64' b'GZMB' b'C1q' b'CD40' b'CD45RA'
 b'CD138' b'IL7R' b'IgM' b'PDGFRb' b'aSMA' b'CD8' b'CD4' b'PanCytoK']
<KeysViewHDF5 ['cells', 'intensity', 'meta']>
<KeysViewHDF5 ['C1q', 'CD103', 'CD11c', 'CD134', 'CD138', 'CD20', 'CD31', 'CD3e', 'CD4', 'CD40', 'CD40LG', 'CD45', 'CD45RA', 'CD45RO', 'CD64', 'CD68', 'CD69', 'CD8', 'CD80', 'CD89', 'CXCL13', 'CXCR5', 'DAPI', 'FOXP3', 'GZMB', 'HLA-DR', 'IL7R', 'IgA', 'IgG', 'IgM', 'Ki-67', 'LAG3', 'OX40L', 'PD-1', 'PD-L1', 'PDGFRb', 'PNaD', 'PanCytoK', 'TIM3', 'aSMA']>


In [3]:
sample_x = tf.zeros((8, 64, 64, len(use_channels)), dtype=tf.float32)
ae_model = Autoencoder(input_shape=sample_x.shape[1:])
y, z_g = ae_model(sample_x, return_g=True)
print(y.shape)
print(z_g.shape)
z = ae_model.encode(sample_x)
print(z.shape)

/home/ing/miniconda3/envs/micron2/lib/python3.8/site-packages/tensorflow/python/keras/applications/imagenet_utils.py:331: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 40 input channels.
  warnings.warn('This model usually expects 1 or 3 input channels. '


(8, 64, 64, 40)
(8, 32)
(8, 256)


In [4]:
ae_model.load_weights(f'{outdir}/weights.h5')

In [5]:
def process(x):
    x = tf.cast(x, tf.float32)/255.
    x = tf.transpose(tf.image.per_image_standardization(tf.transpose(x)))
    return x
    
# use_channels=['DAPI', 'CD45', 'PanCytoK']
# use_channels=['DAPI', 'CD45', 'PanCytoK', 'CD31', 'PDGFRb', 'aSMA', 'Ki-67']
dataset = stream_dataset('/dev/shm/dataset.hdf5', use_channels = use_channels)
dataset = (dataset.map(process)
           .batch(64)
          )

z = []
z_L2 = []
for batch in tqdm.tqdm(dataset):
    # batch = tf.image.random_crop(batch, size=(batch.shape[0], 48, 48, batch.shape[-1]))
    z.append(ae_model.encode_g(batch).numpy().copy())
    z_L2.append(ae_model.encode(batch).numpy().copy())
    
z = np.concatenate(z, axis=0)
z_L2 = np.concatenate(z_L2, axis=0)
print(z.shape)
print((z.sum(0)==0).sum())

print(z_L2.shape)
print((z_L2.sum(0)==0).sum())


(146791, 32)
0
(146791, 256)
187


In [6]:
np.save(f'{outdir}/z.npy', z)
np.save(f'{outdir}/z_L2.npy', z_L2)

In [7]:
!ls -lha trained_simclr

total 275M
drwxrwxr-x 2 ing ing 4.0K Nov 16 22:32 .
drwxrwxr-x 4 ing ing 4.0K Nov 17 09:13 ..
-rw-rw-r-- 1 ing ing 114M Nov 17 05:22 weights.h5
-rw-rw-r-- 1 ing ing 144M Nov 17 09:20 z_L2.npy
-rw-rw-r-- 1 ing ing  18M Nov 17 09:20 z.npy
